## Isolation Tests and Skeletons
#### Post Supply lines. RAG Pipeline Concepts.


In [1]:
from pathlib import Path
import sys
import logging

logging.getLogger().setLevel(logging.WARNING)

current = Path.cwd()
for parent in [current] + list(current.parents):
    if parent.name == "ModelPipeline":
        model_root = parent
        break
else:
    raise RuntimeError("Cannot find 'ModelPipeline' root in path tree")

if str(model_root) not in sys.path:
    sys.path.insert(0, str(model_root))

print(f"✓ Model root on sys.path: {model_root}")


METRIC_DATA_JSON = model_root / "finrag_ml_tg1/rag_modules_src/metric_pipeline/data/downloaded_data.json"
DIM_COMPANIES = model_root / "finrag_ml_tg1/data_cache/dimensions/finrag_dim_companies_21.parquet"
DIM_SECTIONS = model_root / "finrag_ml_tg1/data_cache/dimensions/finrag_dim_sec_sections.parquet"
print(f"✓ Metric data JSON path: {METRIC_DATA_JSON}")
print(f"✓ Dimension companies path: {DIM_COMPANIES}")
print(f"✓ Dimension sections path: {DIM_SECTIONS}")

✓ Model root on sys.path: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline
✓ Metric data JSON path: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\rag_modules_src\metric_pipeline\data\downloaded_data.json
✓ Dimension companies path: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_companies_21.parquet
✓ Dimension sections path: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_sec_sections.parquet


In [2]:
## Quick Verifications:

import polars as pl

# Load Stage 3 S3 Vectors parquet
stage3_path = model_root / "finrag_ml_tg1/data_cache/stage3_s3vectors/cohere_1024d/finrag_embeddings_s3vectors_cohere_1024d.parquet"
df = pl.read_parquet(stage3_path)

# Check what section_name values look like
# print("Sample section_name values:")
# print(df.select("section_name").unique().head(10))

# Stage 2 Meta table: ModelPipeline\finrag_ml_tg1\data_cache\meta_embeds\finrag_fact_sentences_meta_embeds.parquet
stage2_path = model_root / "finrag_ml_tg1/data_cache/meta_embeds/finrag_fact_sentences_meta_embeds.parquet"
df_stage2 = pl.read_parquet(stage2_path)
print(f"✓ Loaded Stage 2 Meta parquet: {stage2_path}")

# Stage 2 vector table: ModelPipeline\finrag_ml_tg1\data_cache\embeddings\cohere_1024d\finrag_embeddings_cohere_1024d.parquet
df_stage2_vecs = pl.read_parquet(model_root / "finrag_ml_tg1/data_cache/embeddings/cohere_1024d/finrag_embeddings_cohere_1024d.parquet")
print(f"✓ Loaded Stage 3 S3 Vectors parquet: {stage3_path}")


"""
1. sentenceID is ALWAYS a string composite key
Format: "{cik}_{form}_{year}_section_{section}_{position}"
Example: "0001045810_10-K_2019_section_7_13"
"""

# print column names or schema of both tables.
print("Stage 2 Meta schema:")
print(df_stage2.schema)
print("Stage 2 vector table:")
print(df_stage2_vecs.schema)


✓ Loaded Stage 2 Meta parquet: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\meta_embeds\finrag_fact_sentences_meta_embeds.parquet
✓ Loaded Stage 3 S3 Vectors parquet: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\stage3_s3vectors\cohere_1024d\finrag_embeddings_s3vectors_cohere_1024d.parquet
Stage 2 Meta schema:
Schema([('cik', String), ('cik_int', Int32), ('name', String), ('tickers', List(String)), ('docID', String), ('sentenceID', String), ('section_ID', Int64), ('section_name', String), ('form', String), ('sic', String), ('sentence', String), ('filingDate', String), ('report_year', Int64), ('reportDate', String), ('temporal_bin', String), ('likely_kpi', Boolean), ('has_numbers', Boolean), ('has_comparison', Boolean), ('sample_created_at', Datetime(time_unit='us', time_zone='UTC')), ('last_modified_date', Datetime(time_unit='us', time_zone='UTC')), ('sample_versi

In [4]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL: VISUAL FILTER TESTS - Just run and observe
# ═══════════════════════════════════════════════════════════════════════════

from finrag_ml_tg1.rag_modules_src.rag_pipeline.metadata_filters import MetadataFilterBuilder
from finrag_ml_tg1.rag_modules_src.entity_adapter.entity_adapter import EntityAdapter

adapter = EntityAdapter( company_dim_path=DIM_COMPANIES, section_dim_path=DIM_SECTIONS )
builder = MetadataFilterBuilder(recent_year_threshold=2015)

# ═══════════════════════════════════════════════════════════════════════════
# TEST QUERIES - Cover all filter dimensions
# ═══════════════════════════════════════════════════════════════════════════

test_queries = [
    # Test 1: Multi-company + multi-year + metrics
    "What was NVIDIA's, Apple's, and Microsoft's revenue and net income in 2020, 2021, and 2022?",
    
    # Test 2: Explicit sections + risk topics
    "In Item 1A and Item 7A, what liquidity risks and market risks did Tesla highlight in 2019 and 2021?",
    
    # Test 3: Year range + MD&A + fuzzy names
    "Looking at the MD&A (Item 7), how did Nvidia's gaming division and MSFT discuss operating results from 2017 to 2020?",
    
    # Test 4: Edge case stress (future years, fuzzy tickers, many sections)
    "For EXXON MOBIL, Meta Platforms, and Walmrt, show total assets in Item 1, Item 7, and Item 8 for 2015-2019, 2022, and 2026."
]


# ═══════════════════════════════════════════════════════════════════════════
# RUN TESTS - Simple loop with clean output
# ═══════════════════════════════════════════════════════════════════════════

for i, query in enumerate(test_queries, 1):
    print("\n" + "="*80)
    print(f"TEST {i}")
    print("="*80)
    print(f"\nQuery: {query}\n")
    
    # Extract entities
    entities = adapter.extract(query)
    
    # Build filters
    filtered = builder.build_filters(entities)
    global_f = builder.build_global_filters(entities)
    
    # Print results
    print(f"Companies: {entities.companies.tickers if entities.companies else 'None'}")
    print(f"CIKs: {list(entities.companies.ciks_int) if entities.companies else 'None'}")
    print(f"Years: {entities.years.years if entities.years else 'None'}")
    if entities.years and entities.years.future_years:
        print(f"  ⚠️  Future years detected: {entities.years.future_years}")
        print(f"  Warning: {entities.years.warning}")
    
    # Sections
    if entities.sections:
        sections_list = (
            list(entities.sections.items) if hasattr(entities.sections, 'items') 
            else entities.sections.get('items', []) if isinstance(entities.sections, dict)
            else []
        )
        if sections_list:
            print(f"Sections: {sections_list}")
    
    print("\n" + "-"*80)
    print("FILTERS:")
    print("-"*80)
    print("\nFiltered call:")
    if filtered:
        for key, val in filtered.items():
            print(f"  {key}: {val}")
    else:
        print("  (no filters)")
    
    print("\nGlobal call:")
    if global_f:
        for key, val in global_f.items():
            print(f"  {key}: {val}")
    else:
        print("  (no filters)")


print("\n" + "="*80)
print("ALL TESTS COMPLETE")
print("="*80)

✓ FilterExtractor initialized with 21 companies
  Using: finrag_dim_companies_21.parquet

TEST 1

Query: What was NVIDIA's, Apple's, and Microsoft's revenue and net income in 2020, 2021, and 2022?

Companies: ['AAPL', 'MSFT', 'NVDA']
CIKs: [320193, 789019, 1045810]
Years: [2020, 2021, 2022]

--------------------------------------------------------------------------------
FILTERS:
--------------------------------------------------------------------------------

Filtered call:
  cik_int: {'$in': [320193, 789019, 1045810]}
  report_year: {'$in': [2020, 2021, 2022]}

Global call:
  cik_int: {'$in': [320193, 789019, 1045810]}
  report_year: {'$gte': 2015}

TEST 2

Query: In Item 1A and Item 7A, what liquidity risks and market risks did Tesla highlight in 2019 and 2021?

Companies: ['TSLA']
CIKs: [1318605]
Years: [2019, 2021]

--------------------------------------------------------------------------------
FILTERS:
-----------------------------------------------------------------------------

In [2]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL: TEST CONFIG LOADING (NO AWS CALLS)
# ═══════════════════════════════════════════════════════════════════════════

from finrag_ml_tg1.loaders.ml_config_loader import MLConfig

config = MLConfig()

# Test variant config
variant_cfg = config.get_variant_config()
print("Variant Config:")
print(f"  enabled: {variant_cfg['enabled']}")
print(f"  model_id: {variant_cfg['model_id']}")
print(f"  count: {variant_cfg['count']}")
print(f"  prompt preview: {variant_cfg['prompt_template'][:100]}...")

print("\n" + "="*80 + "\n")

# Test retrieval config
retrieval_cfg = config.get_retrieval_config()
print("Retrieval Config:")
print(f"  top_k_filtered: {retrieval_cfg['top_k_filtered']}")
print(f"  top_k_global: {retrieval_cfg['top_k_global']}")
print(f"  enable_global: {retrieval_cfg['enable_global']}")
print(f"  enable_variants: {retrieval_cfg['enable_variants']}")
print(f"  vector_bucket: {retrieval_cfg['vector_bucket']}")
print(f"  index_name: {retrieval_cfg['index_name']}")



[DEBUG] ✓ AWS credentials loaded from aws_credentials.env
Variant Config:
  enabled: True
  model_id: anthropic.claude-3-haiku-20240307-v1:0
  count: 3
  prompt preview: Rephrase this financial query {count} different ways while preserving the exact intent and entities....


Retrieval Config:
  top_k_filtered: 30
  top_k_global: 18
  enable_global: True
  enable_variants: True
  vector_bucket: finrag-embeddings-s3vectors
  index_name: finrag-sentence-fact-embed-1024d


In [5]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL: TEST VARIANT GENERATION - COMPLEX QUERY
# ═══════════════════════════════════════════════════════════════════════════

from finrag_ml_tg1.loaders.ml_config_loader import MLConfig
from finrag_ml_tg1.rag_modules_src.rag_pipeline.variant_generator import VariantGenerator

# Load config
config = MLConfig()
variant_cfg = config.get_variant_config()
bedrock_client = config.get_bedrock_client()

print(f"Using model: {variant_cfg['model_id']}")
print(f"Enabled: {variant_cfg['enabled']}")
print(f"Count: {variant_cfg['count']}")
print("\n" + "="*80 + "\n")

# Initialize generator
generator = VariantGenerator(variant_cfg, bedrock_client)

# TOUGHER QUERY - Multi-company, sections, time range, narrative focus
query = "In the MD&A and Risk Factors sections, how did NVIDIA and Microsoft discuss their AI strategy, competitive positioning, and supply chain risks between 2020 and 2023?"

print(f"Original Query:\n  {query}\n")
print("="*80)
print("Generating variants...\n")

variants = generator.generate(query)

if variants:
    print(f"Generated {len(variants)} variants:\n")
    for i, variant in enumerate(variants, 1):
        print(f"{i}. {variant}")
        print()
else:
    print("No variants generated (check logs)")

print("="*80)

# Verify preservation of entities
print("\nEntity Preservation Check:")
print(f"  Companies mentioned: NVIDIA, Microsoft")
print(f"  Sections referenced: MD&A (Item 7), Risk Factors (Item 1A)")
print(f"  Time period: 2020-2023")
print(f"  Topics: AI strategy, competitive positioning, supply chain risks")
print("\n  → Variants should preserve ALL of these elements")

[DEBUG] ✓ AWS credentials loaded from aws_credentials.env
Using model: us.anthropic.claude-haiku-4-5-20251001-v1:0
Enabled: True
Count: 3


Original Query:
  In the MD&A and Risk Factors sections, how did NVIDIA and Microsoft discuss their AI strategy, competitive positioning, and supply chain risks between 2020 and 2023?

Generating variants...

Generated 3 variants:

1. Between 2020 and 2023, what did NVIDIA and Microsoft say about their AI strategy, competitive positioning, and supply chain risks in their MD&A and Risk Factors sections?

2. How did NVIDIA and Microsoft address AI strategy, competitive positioning, and supply chain risks in the MD&A and Risk Factors sections from 2020 through 2023?

3. In their MD&A and Risk Factors disclosures spanning 2020 to 2023, how did NVIDIA and Microsoft characterize their AI strategy, competitive standing, and supply chain risk exposure?


Entity Preservation Check:
  Companies mentioned: NVIDIA, Microsoft
  Sections referenced: MD&A (Item 7

``` 
ALL looks good.
✅ Zero entity loss - All companies, sections, years, and topics preserved
✅ Meaningful variation - Each variant has different structure/phrasing
✅ Semantic equivalence - All variants ask the same fundamental question
✅ Natural language - No awkward phrasing or grammatical errors 
```
# ═══════════════════════════════════════════════════════════════════════════

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# DIAGNOSTIC: Minimal S3 Vectors Test
# Test if S3VectorsRetriever can get ANY results at all
# ═══════════════════════════════════════════════════════════════════════════

from finrag_ml_tg1.loaders.ml_config_loader import MLConfig
from finrag_ml_tg1.rag_modules_src.rag_pipeline.s3_retriever import S3VectorsRetriever
from finrag_ml_tg1.rag_modules_src.utilities.query_embedder_v2 import (
    EmbeddingRuntimeConfig,
    QueryEmbedderV2
)
from finrag_ml_tg1.rag_modules_src.entity_adapter.entity_adapter import EntityAdapter
import numpy as np

# ═══════════════════════════════════════════════════════════════════════════
# SETUP
# ═══════════════════════════════════════════════════════════════════════════

config = MLConfig()
bedrock_client = config.get_bedrock_client()

# Simple query
query = "What was NVIDIA's revenue in 2021?"

# Get embedding
adapter = EntityAdapter(DIM_COMPANIES, DIM_SECTIONS)
entities = adapter.extract(query)

embedding_cfg_dict = config.cfg["embedding"]
runtime_cfg = EmbeddingRuntimeConfig.from_ml_config(embedding_cfg_dict)
embedder = QueryEmbedderV2(runtime_cfg, boto_client=bedrock_client)

embedding = embedder.embed_query(query, entities)

print(f"Query: {query}")
print(f"Embedding dims: {len(embedding)}")
print(f"Embedding preview: {embedding[:5]}")
print("\n" + "="*80)

# ═══════════════════════════════════════════════════════════════════════════
# TEST 1: No filters at all (open retrieval)
# ═══════════════════════════════════════════════════════════════════════════

print("\n TEST 1: Open Retrieval (No Filters)\n")

retrieval_cfg = config.get_retrieval_config()
retrieval_cfg['min_similarity'] = 0.0  # Accept everything
retrieval_cfg['enable_global'] = False  # Only one call for simplicity

retriever = S3VectorsRetriever(
    retrieval_config=retrieval_cfg,
    aws_access_key_id=config.aws_access_key,
    aws_secret_access_key=config.aws_secret_key,
    region=config.region
)

try:
    bundle = retriever.retrieve(
        base_embedding=embedding,
        variant_embeddings=[],
        filtered_filters=None,  # ← NO FILTERS
        global_filters=None,
        base_query=query
    )
    
    print(f"✅ Retrieval succeeded")
    print(f"   Filtered hits: {len(bundle.filtered_hits)}")
    print(f"   Global hits: {len(bundle.global_hits)}")
    print(f"   Union hits: {len(bundle.union_hits)}")
    
    if bundle.union_hits:
        print(f"\n📋 First 5 hits:")
        for i, hit in enumerate(bundle.union_hits[:5], 1):
            print(f"   {i}. sentence_id={hit.sentence_id}, "
                  f"similarity={hit.similarity_score():.3f}, "
                  f"cik={hit.cik_int}, year={hit.report_year}")
    else:
        print("\n❌ No hits returned even with no filters")
        
except Exception as e:
    print(f"❌ Exception during retrieval: {e}")
    import traceback
    traceback.print_exc()

# ═══════════════════════════════════════════════════════════════════════════
# TEST 2: Simple filter (single CIK)
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print(" TEST 2: Simple Filter (CIK only)\n")

simple_filter = {"cik_int": {"$eq": 1045810}}  # NVIDIA

try:
    bundle2 = retriever.retrieve(
        base_embedding=embedding,
        variant_embeddings=[],
        filtered_filters=simple_filter,
        global_filters=None,
        base_query=query
    )
    
    print(f"✅ Retrieval succeeded")
    print(f"   Union hits: {len(bundle2.union_hits)}")
    
    if bundle2.union_hits:
        print(f"\n📋 First 5 hits:")
        for i, hit in enumerate(bundle2.union_hits[:5], 1):
            print(f"   {i}. sentence_id={hit.sentence_id}, "
                  f"similarity={hit.similarity_score():.3f}, "
                  f"cik={hit.cik_int}, year={hit.report_year}")
    else:
        print("\n❌ No hits with simple CIK filter")
        
except Exception as e:
    print(f"❌ Exception during retrieval: {e}")
    import traceback
    traceback.print_exc()

# ═══════════════════════════════════════════════════════════════════════════
# TEST 3: Check actual S3 client directly (bypass our class)
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print(" TEST 3: Direct S3 Vectors Call (Bypass Retriever Class)\n")

import boto3

s3v = boto3.client(
    's3vectors',
    region_name=config.region,
    aws_access_key_id=config.aws_access_key,
    aws_secret_access_key=config.aws_secret_key
)

try:
    response = s3v.query_vectors(
        vectorBucketName=retrieval_cfg['vector_bucket'],
        indexName=retrieval_cfg['index_name'],
        queryVector={"float32": embedding},
        topK=10,
        returnMetadata=True,
        returnDistance=True
        # No filter - open retrieval
    )
    
    vectors = response.get('vectors', [])
    print(f"✅ Direct S3 call succeeded")
    print(f"   Returned {len(vectors)} vectors")
    
    if vectors:
        print(f"\n📋 First 3 raw responses:")
        for i, vec in enumerate(vectors[:3], 1):
            md = vec.get('metadata', {})
            dist = vec.get('distance', 'N/A')
            print(f"   {i}. distance={dist}, metadata keys={list(md.keys())}")
            print(f"      sentenceID={md.get('sentenceID')}, cik_int={md.get('cik_int')}")
    else:
        print("\n❌ Direct S3 call returned 0 vectors")
        print(f"   Full response: {response}")
        
except Exception as e:
    print(f"❌ Direct S3 call failed: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*80)
print("✅ DIAGNOSTIC COMPLETE")
print("="*80)





[DEBUG] ✓ AWS credentials loaded from aws_credentials.env
✓ FilterExtractor initialized with 21 companies
  Using: finrag_dim_companies_21.parquet
Query: What was NVIDIA's revenue in 2021?
Embedding dims: 1024
Embedding preview: [-0.012878418, -0.0066223145, 0.013366699, 0.010498047, 0.008178711]


🧪 TEST 1: Open Retrieval (No Filters)

✅ Retrieval succeeded
   Filtered hits: 30
   Global hits: 0
   Union hits: 30

📋 First 5 hits:
   1. sentence_id=0001045810_10-K_2019_section_7_13, similarity=0.829, cik=1045810, year=2019
   2. sentence_id=0001045810_10-K_2016_section_7_15, similarity=0.823, cik=1045810, year=2016
   3. sentence_id=0001045810_10-K_2020_section_7_13, similarity=0.820, cik=1045810, year=2020
   4. sentence_id=0001045810_10-K_2019_section_15_451, similarity=0.815, cik=1045810, year=2019
   5. sentence_id=0001045810_10-K_2018_section_15_478, similarity=0.815, cik=1045810, year=2018

🧪 TEST 2: Simple Filter (CIK only)

✅ Retrieval succeeded
   Union hits: 30

📋 First 5 hit

In [ ]:
# # Test different filter patterns to find what S3 Vectors accepts

# # Pattern A: Single section (works in diagnostic test)
# filter_A = {
#     'cik_int': {'$in': [789019, 1045810]},
#     'report_year': {'$in': [2020, 2023]},
#     'section_name': {'$eq': 'ITEM_7'}  # Single value
# }

# # Pattern B: Multiple sections with $in
# filter_B = {
#     'cik_int': {'$in': [789019, 1045810]},
#     'report_year': {'$in': [2020, 2023]},
#     'section_name': {'$in': ['ITEM_7', 'ITEM_1A']}  # Multiple - MIGHT FAIL
# }

# # Pattern C: OR wrapper (AWS S3 Vectors syntax)
# filter_C = {
#     '$and': [
#         {'cik_int': {'$in': [789019, 1045810]}},
#         {'report_year': {'$in': [2020, 2023]}},
#         {'$or': [
#             {'section_name': {'$eq': 'ITEM_7'}},
#             {'section_name': {'$eq': 'ITEM_1A'}}
#         ]}
#     ]
# }

In [4]:
# ═══════════════════════════════════════════════════════════════════════════
# QUICK SANITY TEST: Filtered + Global with Complex Query
# Purpose: Verify filter syntax fix (should take <30 seconds)
# ═══════════════════════════════════════════════════════════════════════════

from finrag_ml_tg1.loaders.ml_config_loader import MLConfig
from finrag_ml_tg1.rag_modules_src.entity_adapter.entity_adapter import EntityAdapter
from finrag_ml_tg1.rag_modules_src.utilities.query_embedder_v2 import (
    EmbeddingRuntimeConfig,
    QueryEmbedderV2
)
from finrag_ml_tg1.rag_modules_src.rag_pipeline.metadata_filters import MetadataFilterBuilder
from finrag_ml_tg1.rag_modules_src.rag_pipeline.s3_retriever import S3VectorsRetriever

# ═══════════════════════════════════════════════════════════════════════════
# SETUP
# ═══════════════════════════════════════════════════════════════════════════

config = MLConfig()
bedrock_client = config.get_bedrock_client()

adapter = EntityAdapter(
    company_dim_path=DIM_COMPANIES,
    section_dim_path=DIM_SECTIONS
)

embedding_cfg = config.cfg["embedding"]
runtime_cfg = EmbeddingRuntimeConfig.from_ml_config(embedding_cfg)
embedder = QueryEmbedderV2(runtime_cfg, boto_client=bedrock_client)

filter_builder = MetadataFilterBuilder(recent_year_threshold=2015)

retrieval_cfg = config.get_retrieval_config()
retrieval_cfg['min_similarity'] = 0.0  # Accept all results for now

retriever = S3VectorsRetriever(
    retrieval_config=retrieval_cfg,
    aws_access_key_id=config.aws_access_key,
    aws_secret_access_key=config.aws_secret_key,
    region=config.region
)

# ═══════════════════════════════════════════════════════════════════════════
# TEST QUERY
# ═══════════════════════════════════════════════════════════════════════════

query = "In the MD&A and Risk Factors sections, how did NVIDIA and Microsoft discuss their AI strategy, competitive positioning, and supply chain risks between 2020 and 2023?"

print(f"Query: {query}\n")
print("="*80)

# ────────────────────────────────────────────────────────────────────────────
# STEP 1: Extract Entities
# ────────────────────────────────────────────────────────────────────────────

print("\n📋 STEP 1: Entity Extraction\n")

entities = adapter.extract(query)

print(f"Companies: {entities.companies.tickers}")
print(f"CIKs: {list(entities.companies.ciks_int)}")
print(f"Years: {entities.years.years}")
print(f"Sections: {entities.sections.items if hasattr(entities.sections, 'items') else 'N/A'}")

# ────────────────────────────────────────────────────────────────────────────
# STEP 2: Build Filters
# ────────────────────────────────────────────────────────────────────────────

print("\n" + "="*80)
print("🔧 STEP 2: Build Filters\n")

filtered_filters = filter_builder.build_filters(entities)
global_filters = filter_builder.build_global_filters(entities)

print("Filtered call filter:")
print(filtered_filters)

print("\nGlobal call filter:")
print(global_filters)

# ────────────────────────────────────────────────────────────────────────────
# STEP 3: Generate Embedding
# ────────────────────────────────────────────────────────────────────────────

print("\n" + "="*80)
print("🧠 STEP 3: Query Embedding\n")

embedding = embedder.embed_query(query, entities)
print(f"Embedding: {len(embedding)}D vector")

# ────────────────────────────────────────────────────────────────────────────
# STEP 4: RETRIEVE (Filtered + Global)
# ────────────────────────────────────────────────────────────────────────────

print("\n" + "="*80)
print("🚀 STEP 4: Retrieval (Filtered + Global)\n")

try:
    bundle = retriever.retrieve(
        base_embedding=embedding,
        variant_embeddings=[],  # No variants for sanity test
        filtered_filters=filtered_filters,
        global_filters=global_filters,
        base_query=query
    )
    
    print("✅ RETRIEVAL SUCCEEDED\n")
    print(f"Filtered hits: {len(bundle.filtered_hits)}")
    print(f"Global hits: {len(bundle.global_hits)}")
    print(f"Union (deduplicated): {len(bundle.union_hits)}")
    
    # ────────────────────────────────────────────────────────────────────
    # Show first 5 results
    # ────────────────────────────────────────────────────────────────────
    if bundle.union_hits:
        print("\n" + "─"*80)
        print("📊 First 5 Results:\n")
        
        for i, hit in enumerate(bundle.union_hits[:5], 1):
            print(f"{i}. sentence_id: {hit.sentence_id}")
            print(f"   similarity: {hit.similarity_score():.3f}")
            print(f"   cik: {hit.cik_int}, year: {hit.report_year}, section: {hit.section_name}")
            print(f"   source: {hit.source}, embedding_id: {hit.embedding_id[:40]}...")
            print()
    
    # ────────────────────────────────────────────────────────────────────
    # Validate results match filters
    # ────────────────────────────────────────────────────────────────────
    print("─"*80)
    print("🔍 Filter Validation:\n")
    
    expected_ciks = {789019, 1045810}  # MSFT, NVDA
    expected_years = {2020, 2023}
    expected_sections = {"ITEM_7", "ITEM_1A"}
    
    actual_ciks = {h.cik_int for h in bundle.filtered_hits}
    actual_years = {h.report_year for h in bundle.filtered_hits}
    actual_sections = {h.section_name for h in bundle.filtered_hits}
    
    print(f"Expected CIKs: {expected_ciks}")
    print(f"Actual CIKs: {actual_ciks}")
    print(f"  ✅ Match!" if actual_ciks.issubset(expected_ciks) else f"  ❌ Mismatch!")
    
    print(f"\nExpected Years: {expected_years}")
    print(f"Actual Years: {actual_years}")
    print(f"  ✅ Match!" if actual_years.issubset(expected_years) else f"  ❌ Mismatch!")
    
    print(f"\nExpected Sections: {expected_sections}")
    print(f"Actual Sections: {actual_sections}")
    print(f"  ✅ Match!" if actual_sections.issubset(expected_sections) else f"  ❌ Mismatch!")
    
except Exception as e:
    print(f"❌ RETRIEVAL FAILED\n")
    print(f"Error: {e}")
    
    import traceback
    traceback.print_exc()

print("\n" + "="*80)
print("✅ SANITY TEST COMPLETE")
print("="*80)
# ═══════════════════════════════════════════════════════════════════════════



[DEBUG] ✓ AWS credentials loaded from aws_credentials.env
✓ FilterExtractor initialized with 21 companies
  Using: finrag_dim_companies_21.parquet
Query: In the MD&A and Risk Factors sections, how did NVIDIA and Microsoft discuss their AI strategy, competitive positioning, and supply chain risks between 2020 and 2023?


📋 STEP 1: Entity Extraction

Companies: ['MSFT', 'NVDA']
CIKs: [789019, 1045810]
Years: [2020, 2023]
Sections: N/A

🔧 STEP 2: Build Filters

Filtered call filter:
{'$and': [{'cik_int': {'$in': [789019, 1045810]}}, {'report_year': {'$in': [2020, 2023]}}, {'$or': [{'section_name': {'$eq': 'ITEM_1A'}}, {'section_name': {'$eq': 'ITEM_7'}}]}]}

Global call filter:
{'cik_int': {'$in': [789019, 1045810]}, 'report_year': {'$gte': 2015}}

🧠 STEP 3: Query Embedding

Embedding: 1024D vector

🚀 STEP 4: Retrieval (Filtered + Global)



Global call failed for variant 0: An error occurred (ValidationException) when calling the QueryVectors operation: Invalid query filter


✅ RETRIEVAL SUCCEEDED

Filtered hits: 30
Global hits: 0
Union (deduplicated): 30

────────────────────────────────────────────────────────────────────────────────
📊 First 5 Results:

1. sentence_id: 0001045810_10-K_2020_section_1A_204
   similarity: 0.737
   cik: 1045810, year: 2020, section: ITEM_1A
   source: filtered, embedding_id: bedrock_cohere_v4_1024d_20251109_1407...

2. sentence_id: 0001045810_10-K_2020_section_1A_168
   similarity: 0.701
   cik: 1045810, year: 2020, section: ITEM_1A
   source: filtered, embedding_id: bedrock_cohere_v4_1024d_20251109_1407...

3. sentence_id: 0000789019_10-K_2020_section_1A_218
   similarity: 0.701
   cik: 789019, year: 2020, section: ITEM_1A
   source: filtered, embedding_id: bedrock_cohere_v4_1024d_20251109_1355...

4. sentence_id: 0000789019_10-K_2020_section_1A_163
   similarity: 0.699
   cik: 789019, year: 2020, section: ITEM_1A
   source: filtered, embedding_id: bedrock_cohere_v4_1024d_20251109_1355...

5. sentence_id: 0000789019_10-K_202

In [3]:
## ═══════════════════════════════════════════════════════════════════════════
## Similarity threshold impact analysis testing. 
## ═══════════════════════════════════════════════════════════════════════════
# Research Question: Should we filter by min_similarity given S3 Vectors' top-K?
# ═══════════════════════════════════════════════════════════════════════════

from finrag_ml_tg1.loaders.ml_config_loader import MLConfig
from finrag_ml_tg1.rag_modules_src.entity_adapter.entity_adapter import EntityAdapter
from finrag_ml_tg1.rag_modules_src.utilities.query_embedder_v2 import (
    EmbeddingRuntimeConfig,
    QueryEmbedderV2
)
from finrag_ml_tg1.rag_modules_src.rag_pipeline.metadata_filters import MetadataFilterBuilder
from finrag_ml_tg1.rag_modules_src.rag_pipeline.s3_retriever import S3VectorsRetriever
from finrag_ml_tg1.rag_modules_src.rag_pipeline.variant_generator import VariantGenerator
import numpy as np
import polars as pl

# ═══════════════════════════════════════════════════════════════════════════
# SETUP
# ═══════════════════════════════════════════════════════════════════════════

config = MLConfig()

# Initialize all components
adapter = EntityAdapter(
    company_dim_path=DIM_COMPANIES,
    section_dim_path=DIM_SECTIONS
)

embedding_cfg_dict = config.cfg["embedding"]
runtime_cfg = EmbeddingRuntimeConfig.from_ml_config(embedding_cfg_dict)
bedrock_client = config.get_bedrock_client()
embedder = QueryEmbedderV2(runtime_cfg, boto_client=bedrock_client)

filter_builder = MetadataFilterBuilder(recent_year_threshold=2015)

variant_cfg = config.get_variant_config()
variant_generator = VariantGenerator(variant_cfg, bedrock_client)

# Test query
query = "In the MD&A and Risk Factors sections, how did NVIDIA and Microsoft discuss their AI strategy, competitive positioning, and supply chain risks between 2020 and 2023?"

print(f"Query: {query}\n")
print("="*80)

# ═══════════════════════════════════════════════════════════════════════════
# STEP 1: Extract Entities & Build Filters
# ═══════════════════════════════════════════════════════════════════════════

print("\n🔍 STEP 1: Entity Extraction\n")
entities = adapter.extract(query)

print(f"Companies: {entities.companies.tickers}")
print(f"CIKs: {list(entities.companies.ciks_int)}")
print(f"Years: {entities.years.years}")

filtered_filters = filter_builder.build_filters(entities)
global_filters = filter_builder.build_global_filters(entities)

print(f"\nFiltered filters: {filtered_filters}")
print(f"Global filters: {global_filters}")

# ═══════════════════════════════════════════════════════════════════════════
# STEP 2: Generate Embedding
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("🧠 STEP 2: Query Embedding\n")

embedding = embedder.embed_query(query, entities)
print(f"Embedding: {len(embedding)}D vector, preview: {embedding[:5]}")

# ═══════════════════════════════════════════════════════════════════════════
# EXPERIMENT: Test Different Similarity Thresholds
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print(" EXPERIMENT: Similarity Threshold Impact")
print("="*80)

thresholds = [0.0, 0.2, 0.3, 0.4, 0.5]
results = {}

for threshold in thresholds:
    print(f"\n{'─'*80}")
    print(f"Testing threshold: {threshold}")
    print('─'*80)
    
    # Create retriever with this threshold
    retrieval_cfg = config.get_retrieval_config()
    retrieval_cfg['min_similarity'] = threshold
    retrieval_cfg['enable_variants'] = False  # Disable variants for clean test
    
    retriever = S3VectorsRetriever(
        retrieval_config=retrieval_cfg,
        aws_access_key_id=config.aws_access_key,
        aws_secret_access_key=config.aws_secret_key,
        region=config.region
    )
    
    # Retrieve (base query only, no variants)
    bundle = retriever.retrieve(
        base_embedding=embedding,
        variant_embeddings=[],
        filtered_filters=filtered_filters,
        global_filters=global_filters,
        base_query=query
    )
    
    # Collect statistics
    filtered_count = len(bundle.filtered_hits)
    global_count = len(bundle.global_hits)
    union_count = len(bundle.union_hits)
    
    # Distance distribution
    distances = [h.distance for h in bundle.union_hits]
    similarities = [h.similarity_score() for h in bundle.union_hits]
    
    results[threshold] = {
        'filtered_count': filtered_count,
        'global_count': global_count,
        'union_count': union_count,
        'distances': distances,
        'similarities': similarities,
        'bundle': bundle
    }
    
    print(f"  Filtered hits: {filtered_count}")
    print(f"  Global hits: {global_count}")
    print(f"  Union (unique): {union_count}")
    
    if similarities:
        print(f"  Similarity range: [{min(similarities):.3f}, {max(similarities):.3f}]")
        print(f"  Mean similarity: {np.mean(similarities):.3f}")
        print(f"  Median similarity: {np.median(similarities):.3f}")
    else:
        print("  No hits passed threshold")

# ═══════════════════════════════════════════════════════════════════════════
# ANALYSIS: Summary Table
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("📊 SUMMARY TABLE")
print("="*80 + "\n")

summary_rows = []
for threshold in thresholds:
    r = results[threshold]
    summary_rows.append({
        'threshold': threshold,
        'union_hits': r['union_count'],
        'filtered_hits': r['filtered_count'],
        'global_hits': r['global_count'],
        'min_sim': min(r['similarities']) if r['similarities'] else None,
        'max_sim': max(r['similarities']) if r['similarities'] else None,
        'mean_sim': np.mean(r['similarities']) if r['similarities'] else None,
        'rejected': results[0.0]['union_count'] - r['union_count']  # vs no filter
    })

summary_df = pl.DataFrame(summary_rows)
print(summary_df)

# ═══════════════════════════════════════════════════════════════════════════
# DEEP DIVE: Inspect Rejected Hits at threshold=0.3
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("🔬 DEEP DIVE: What Gets Rejected at threshold=0.3?")
print("="*80 + "\n")

all_hits = results[0.0]['bundle'].union_hits  # No filtering
filtered_hits = results[0.3]['bundle'].union_hits  # With 0.3 threshold

# Find rejected hits
all_sentence_ids = {h.sentence_id for h in all_hits}
kept_sentence_ids = {h.sentence_id for h in filtered_hits}
rejected_sentence_ids = all_sentence_ids - kept_sentence_ids

if rejected_sentence_ids:
    rejected_hits = [h for h in all_hits if h.sentence_id in rejected_sentence_ids]
    
    print(f"Rejected {len(rejected_hits)} hits:\n")
    
    rejected_data = []
    for h in rejected_hits:
        rejected_data.append({
            'sentence_id': h.sentence_id,
            'similarity': h.similarity_score(),
            'distance': h.distance,
            'cik': h.cik_int,
            'year': h.report_year,
            'section': h.sec_item_canonical,
            'source': h.source
        })
    
    rejected_df = pl.DataFrame(rejected_data).sort('similarity', descending=True)
    print(rejected_df)
    
    print(f"\nRejected similarity range: [{min(h.similarity_score() for h in rejected_hits):.3f}, {max(h.similarity_score() for h in rejected_hits):.3f}]")
else:
    print("No hits rejected at threshold=0.3")

# ═══════════════════════════════════════════════════════════════════════════
# VISUALIZATION: Distance Distribution
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("📈 DISTANCE DISTRIBUTION (threshold=0.0)")
print("="*80 + "\n")

all_sims = results[0.0]['similarities']
if all_sims:
    # Simple ASCII histogram
    bins = [0.0, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    hist, _ = np.histogram(all_sims, bins=bins)
    
    print("Similarity distribution:")
    for i, (low, high) in enumerate(zip(bins[:-1], bins[1:])):
        bar = '█' * int(hist[i] * 50 / max(hist)) if max(hist) > 0 else ''
        print(f"  [{low:.1f}-{high:.1f}): {hist[i]:3d} {bar}")

print("\n" + "="*80)
print("✅ EXPERIMENT COMPLETE")
print("="*80)

[DEBUG] ✓ AWS credentials loaded from aws_credentials.env
✓ FilterExtractor initialized with 21 companies
  Using: finrag_dim_companies_21.parquet
Query: In the MD&A and Risk Factors sections, how did NVIDIA and Microsoft discuss their AI strategy, competitive positioning, and supply chain risks between 2020 and 2023?


🔍 STEP 1: Entity Extraction

Companies: ['MSFT', 'NVDA']
CIKs: [789019, 1045810]
Years: [2020, 2023]

Filtered filters: {'$and': [{'cik_int': {'$in': [789019, 1045810]}}, {'report_year': {'$in': [2020, 2023]}}, {'$or': [{'section_name': {'$eq': 'ITEM_1A'}}, {'section_name': {'$eq': 'ITEM_7'}}]}]}
Global filters: {'$and': [{'cik_int': {'$in': [789019, 1045810]}}, {'report_year': {'$gte': 2015}}]}

🧠 STEP 2: Query Embedding

Embedding: 1024D vector, preview: [0.07470703, -0.045410156, 0.008178711, 0.057128906, -0.018554688]

 EXPERIMENT: Similarity Threshold Impact

────────────────────────────────────────────────────────────────────────────────
Testing threshold: 0.0
───